In [ ]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm

In [ ]:
with open('subway_name','rb') as fr:
    data = pickle.load(fr)

In [ ]:
st = pd.DataFrame(columns=['subway_name','위도','경도'])
st['subway_name'] = sum(list(data.values()),[])
val = pd.DataFrame(st['subway_name'].value_counts()).reset_index().rename(columns={'index':'subway_name','subway_name':'환승역 수'})
st = st.drop_duplicates().reset_index(drop=True)
st = pd.merge(st,val,how='inner',on='subway_name')

In [ ]:
st['subway_name'] = st['subway_name'].apply(lambda x : re.sub(r'\([^)]*\)','',x)) #괄호 포함 글자 제거

In [ ]:
def get_location():
    
    subway = np.array(st['subway_name'])
    
    for i in tqdm(subway):
        
        url = "https://dapi.kakao.com/v2/local/search/keyword.json"

        queryString = {'query' : i}
        header = {'Authorization': 'KakaoAK 660c198b2093e22460c08a028177ac18'} #rest api

        response = requests.get(url, headers=header, params=queryString)
        tokens = response.json()
        
        try: 
            lat = tokens['documents'][0]['x']
            lng = tokens['documents'][0]['y']
            st.loc[st['subway_name'].isin([i]),'위도'] = lat
            st.loc[st['subway_name'].isin([i]),'경도'] = lng
        except:
            print(i)
            pass

In [ ]:
get_location()

In [ ]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
st.to_csv(os.path.join(path, "subway.csv"),index=False)